In [1]:
#load in packages 
import numpy as np
import pandas as pd

#regex
import re

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering

from scipy.cluster.hierarchy import dendrogram

import matplotlib.pyplot as plt

In [2]:
items_df = pd.read_csv("data/input/items_2025.csv")
prices_df = pd.read_csv("data/input/prices_2025_cross_section.csv")

In [3]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117580 entries, 0 to 117579
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   store_id            117580 non-null  int64 
 1   product_id          117580 non-null  int64 
 2   product_title       117580 non-null  object
 3   product_url         117580 non-null  object
 4   product_breadcrumb  117580 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.5+ MB


In [4]:
items_df.shape

(117580, 5)

In [5]:
items_df.head(20)

,store_id,product_id,product_title,product_url,product_breadcrumb
0,3,910001819336,Ella's Kitchen Organic Strawberry and Banana M...,http://groceries.asda.com/product/baby-toddler...,"Baby, Toddler & Kids, Baby, Toddler & Kids Sna..."
1,3,910001818776,Ella's Kitchen Organic Tomato and Leek Melty P...,http://groceries.asda.com/product/baby-toddler...,"Baby, Toddler & Kids, Baby, Toddler & Kids Sna..."
2,3,910001817408,Regal Bakery Classic Fairy Cakes,http://groceries.asda.com/product/indian--asia...,"World Food, Indian & Asian Food, Asian Dessert..."
3,3,910001817566,Innocent Summer Fruits Juice,http://groceries.asda.com/product/chilled-juic...,"['Chilled Food', 'Chilled Juice & Smoothies', ..."
4,3,910001821997,Bulldog Skincare for Men Original Face Wash,http://groceries.asda.com/product/mens-toiletr...,"Toiletries & Beauty, Men's Toiletries, Men's S..."
5,3,910001823893,J2O Spritz Pear & Raspberry Sparkling Juice Drink,http://groceries.asda.com/product/tonic-water-...,"Drinks, Tonic Water & Mixers, Adult Soft Drink..."
6,3,910001820433,Copella Cloudy Apple Fruit Juice,http://groceries.asda.com/product/sandwiches-s...,"Chilled Food, Sandwiches, Snacks & Drinks, San..."
7,3,910001816585,Nando's Peri-Peri Sauce Hot,http://groceries.asda.com/product/condiments--...,"Food Cupboard, Condiments & Cooking Ingredient..."
8,3,910001816627,ASDA Extra Special Chardonnay,http://groceries.asda.com/product/wine/asda-ex...,"Beer, Wine & Spirits, Wine, White Wine, Chardo..."
9,3,910001816360,ASDA Extra Special Soave Classico,http://groceries.asda.com/product/wine/asda-ex...,"Beer, Wine & Spirits, Wine, White Wine, All Ot..."


In [6]:
items_df["product_breadcrumb"]

0         Baby, Toddler & Kids, Baby, Toddler & Kids Sna...
1         Baby, Toddler & Kids, Baby, Toddler & Kids Sna...
2         World Food, Indian & Asian Food, Asian Dessert...
3         ['Chilled Food', 'Chilled Juice & Smoothies', ...
4         Toiletries & Beauty, Men's Toiletries, Men's S...
                                ...                        
117575                  ['Groceries/Frozen/Fish & seafood']
117576    Toiletries & Beauty > Dental Care > Toothpaste...
117577    ['Health & Beauty', "Men's Toiletries", 'Showe...
117578    ['Groceries/Food cupboard/Stock up the cupboar...
117579    Groceries > Fresh & Chilled > Fresh Pizza & Ga...
Name: product_breadcrumb, Length: 117580, dtype: object

In [7]:
prices_df.head(15)

,date,store_id,product_id,price,unit_price,loyalty_price,original_price
0,2025-01-01,1,3.253248e+08,4.99,4.99 / each,NaN,NaN
1,2025-01-01,3,1.000383e+12,11.00,£11.00/each,NaN,NaN
2,2025-01-01,2,7.621751e+06,1.80,£8.57 / kg,NaN,1.8
3,2025-01-01,1,3.252527e+08,144.99,144.99 / each,NaN,NaN
4,2025-01-01,2,8.163367e+06,1.30,£1.30 / 100g,NaN,1.3
5,2025-01-01,2,8.201977e+06,1.90,£7.60 / kg,NaN,1.9
6,2025-01-01,1,3.165419e+08,8.50,36.96 / kg,NaN,NaN
7,2025-01-01,1,3.250867e+08,14.99,14.99 / each,NaN,NaN
8,2025-01-01,2,8.199674e+06,1.30,£1.44 / 100g,NaN,1.3
9,2025-01-01,1,3.253245e+08,6.99,6.99 / each,NaN,NaN


In [8]:
prices_df["date"]

0         2025-01-01
1         2025-01-01
2         2025-01-01
3         2025-01-01
4         2025-01-01
             ...    
117548    2025-02-12
117549    2025-02-12
117550    2025-02-12
117551    2025-02-12
117552    2025-02-12
Name: date, Length: 117553, dtype: object

In [9]:
#prices_df = prices_df.set_index("product_id")
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117553 entries, 0 to 117552
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            117553 non-null  object 
 1   store_id        117553 non-null  int64  
 2   product_id      117553 non-null  float64
 3   price           117553 non-null  float64
 4   unit_price      93485 non-null   object 
 5   loyalty_price   7475 non-null    float64
 6   original_price  24971 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 6.3+ MB


In [10]:
print(list(prices_df["unit_price"].unique()))

['4.99 / each', '£11.00/each', '£8.57 / kg', '144.99 / each', '£1.30 / 100g', '£7.60 / kg', '36.96 / kg', '14.99 / each', '£1.44 / 100g', '6.99 / each', '£12.00/each', '27.49 / each', '61.99 / each', '79.99 / each', '8.57 / kg', '3.43 / 100g', '9.99 / each', '25.19 / each', '£17.71/kg', '£10.00/each', '19.99 / each', '75.0p/each', '5.99 / each', '29.99 / each', '44.99 / each', '26.69 / each', '1.5 / 100ml', '6.59 / each', '8.99 / each', '84.99 / each', '49.99 / each', '10.99 / each', '38.99 / each', '9.49 / each', '30 / each', '12.99 / each', '34.99 / each', '21.99 / each', '13.99 / each', '139 / each', '27.99 / each', '16 / each', '16.99 / each', '7.99 / each', '22.99 / each', '20.99 / each', '15.99 / each', '17.99 / each', '11.99 / each', '97.6p/100g', '24.99 / each', '71.99 / each', '25.49 / each', '9.69 / each', '39.99 / each', '17.49 / each', '3.99 / each', '2.5 / 100g', '105.99 / each', '26.99 / each', '28.99 / each', '35.99 / each', '99.99 / each', '10.79 / each', '74.99 / each'

In [11]:
#prices_df = prices_df.dropna(subset = ["loyalty_price"])
#prices_df = prices_df.dropna(subset=["original_price"])

In [12]:
# match datasets
merged_df = items_df.copy()

In [13]:
merged_df = merged_df.merge(prices_df, how='right', on = "product_id", copy=False)
merged_df["store_id"] = merged_df["store_id_x"]
merged_df = merged_df.drop(columns=["store_id_y", "store_id_x"])
merged_df = merged_df.set_index("product_id")

In [14]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117581 entries, 325324827 to 4088600084398
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   product_title       117581 non-null  object 
 1   product_url         117581 non-null  object 
 2   product_breadcrumb  117581 non-null  object 
 3   date                117581 non-null  object 
 4   price               117581 non-null  float64
 5   unit_price          93513 non-null   object 
 6   loyalty_price       7475 non-null    float64
 7   original_price      24979 non-null   float64
 8   store_id            117581 non-null  int64  
dtypes: float64(3), int64(1), object(5)
memory usage: 9.0+ MB


## Extract product category and sub-category

## Clean up breadcrumb column

In [15]:
# clean the breadcrumb string
def clean_breadcrumb(breadcrumb):
    breadcrumb = breadcrumb.replace("'", "").replace("en-GB", "").replace("[","").replace("]","").replace('"','')  # Remove unwanted characters
    breadcrumb = breadcrumb.replace(", ", " > ").replace("/", " > ").replace("-", " ")  # Replace characters


    if breadcrumb.startswith("Marketplace"):
        breadcrumb = breadcrumb.replace("Marketplace >", "", 1)
    
    elif breadcrumb.startswith("Home"):
        breadcrumb = breadcrumb.replace("Home","",1)
    
    elif breadcrumb.startswith("Groceries"):
        breadcrumb = breadcrumb.replace("Groceries","",1)
    return breadcrumb.strip() 

In [16]:
# Apply cleanup to the 'product_breadcrumb' column
merged_df["product_breadcrumb"] = merged_df["product_breadcrumb"].apply(clean_breadcrumb)

In [17]:
merged_df["product_breadcrumb"] = merged_df["product_breadcrumb"].str.replace(r"^>+\s*", "", regex=True)

In [18]:
merged_df.iloc[2342:2354,:]

,product_title,product_url,product_breadcrumb,date,price,unit_price,loyalty_price,original_price,store_id
product_id,,,,,,,,,
113941940,Morrisons White & Green Foliage Dinner Crackers,https://groceries.morrisons.com/products/morri...,Toys & Entertainment > Gift Shop,2025-01-10,6.00,NaN,NaN,NaN,4
113933632,Morrisons Santa Boppers Headband,https://groceries.morrisons.com/products/morri...,Toys & Entertainment > Gift Shop,2025-01-10,1.00,NaN,NaN,NaN,4
113941915,Morrisons Robin Dinner Crackers,https://groceries.morrisons.com/products/morri...,Toys & Entertainment > Gift Shop,2025-01-10,6.00,NaN,NaN,NaN,4
113284185,Barbie Design Your Own Drink Bottle,https://groceries.morrisons.com/products/barbi...,Toys & Entertainment > Christmas Clearance > G...,2025-01-10,6.00,NaN,NaN,NaN,4
114217048,Funkin Black Forest Gateau,https://groceries.morrisons.com/products/funki...,Beer > Wines & Spirits > Pre Mixed Drinks & Co...,2025-01-10,12.50,NaN,NaN,NaN,4
113977627,Fizz Creations Make Your Own White Neon Effec...,https://groceries.morrisons.com/products/fizz-...,Toys & Entertainment > Gift Shop,2025-01-10,6.00,NaN,NaN,NaN,4
114148257,Blue Sky Transformers Pen Fidget,https://groceries.morrisons.com/products/blue-...,Toys & Entertainment > Gift Shop,2025-01-10,4.00,NaN,NaN,NaN,4
113979606,Paladone Arcade Keyring,https://groceries.morrisons.com/products/palad...,Toys & Entertainment > Gift Shop,2025-01-10,5.00,NaN,NaN,NaN,4
113979260,Paladone Minecraft Creeper Stressball,https://groceries.morrisons.com/products/palad...,Toys & Entertainment > Gift Shop,2025-01-10,5.00,NaN,NaN,NaN,4


In [19]:
#Extract product category and subcategory
split_breadcrumbs = merged_df["product_breadcrumb"].str.split(" > ", expand=True)
merged_df["product_category"] = split_breadcrumbs[0]
merged_df["product_subcategory"] = split_breadcrumbs[1]
merged_df["product_subcategory_2"] = split_breadcrumbs[2]

# Convert new columns to lowercase
merged_df["product_breadcrumb"] = merged_df["product_breadcrumb"].str.lower()
merged_df["product_category"] = merged_df["product_category"].str.lower()
merged_df["product_subcategory"] = merged_df["product_subcategory"].str.lower()
merged_df["product_subcategory_2"] = merged_df["product_subcategory_2"].str.lower()


In [20]:
#store id 
n=8
#filter by sore_id
filtered_df = merged_df[merged_df["store_id"] == n]
#randomly choose 20 products to be displayed
random_sample = filtered_df.sample(n=10) #do not set random state!!
random_sample 

,product_title,product_url,product_breadcrumb,date,price,unit_price,loyalty_price,original_price,store_id,product_category,product_subcategory,product_subcategory_2
product_id,,,,,,,,,,,,
538590497820497821,Waitrose Yogurt Dressed Charlotte Potato Salad,https://www.waitrose.com/ecom/products/waitros...,fresh & chilled > fresh salad & herbs > prepar...,2025-02-12,2.25,83.3p/100g,NaN,NaN,8,fresh & chilled,fresh salad & herbs,prepared salads
601504261205261206,Walkers Sensations Thai Sweet Chilli Coated Sh...,https://www.waitrose.com/ecom/products/walkers...,food cupboard > crisps > snacks & nuts > nuts ...,2025-02-12,2.65,£1.77/100g,NaN,NaN,8,food cupboard,crisps,snacks & nuts
807729819079819080,Stripe Ankle Sock Red/ Pink/ Blue 3pk,https://www.waitrose.com/ecom/products/stripe-...,toiletries > health & beauty > tights & socks ...,2025-02-12,10.00,£10 each,NaN,NaN,8,toiletries,health & beauty,tights & socks
515842565825659,Kettle Chips Sea Salt & Balsamic Vinegar,https://www.waitrose.com/ecom/products/kettle-...,food cupboard > crisps > snacks & nuts > singl...,2025-02-12,1.15,£2.88/100g,NaN,NaN,8,food cupboard,crisps,snacks & nuts
505376769459769460,SMA Pro First Infant Milk From Birth Stage 1,https://www.waitrose.com/ecom/products/sma-pro...,baby > child & parent > milk & formula > stage...,2025-02-12,13.00,£16.25/kg,NaN,NaN,8,baby,child & parent,milk & formula
846758730450730451,Waitrose Small Insulated Bag,https://www.waitrose.com/ecom/products/waitros...,home > reusable bags,2025-02-12,2.50,£2.50 each,NaN,NaN,8,home,reusable bags,None
822110791229791230,McVitie's Hobnobs Biscuits The Oaty One,https://www.waitrose.com/ecom/products/mcvitie...,food cupboard > crisps > snacks & nuts > rice ...,2025-02-12,1.65,64.7p/100g,NaN,NaN,8,food cupboard,crisps,snacks & nuts
385363835330835331,Finish Ultimate Deep Clean & Shine Dishwash Ta...,https://www.waitrose.com/ecom/products/finish-...,household > dishwashing > dishwasher tablets,2025-02-12,21.00,22.8p each,NaN,NaN,8,household,dishwashing,dishwasher tablets
865060824215824216,Jacob's Cream Crackers,https://www.waitrose.com/ecom/products/jacobs-...,food cupboard > crisps > snacks & nuts > shari...,2025-02-12,1.35,67.5p/100g,NaN,NaN,8,food cupboard,crisps,snacks & nuts


## Create a filter to keep products of interest

Keep toiletries, household products, kids products 

In [21]:
product_cat_of_interest = ["toiletries", "health", "beauty", "household", "sports" ,"leisure", "laundry", "baby"]

#create a regex pattern to check if any of the strings in list above are contained in rows
pattern = "|".join(product_cat_of_interest)

In [22]:
# Keep only rows that contain any of the categories from the list
filtered_items_df = merged_df[
    merged_df["product_category"].str.contains(pattern, case=False, na=False) |
    merged_df["product_subcategory"].str.contains(pattern, case=False, na=False)
]

In [23]:
print(filtered_items_df.shape)

(24096, 12)


In [24]:
rand_sample_df = filtered_items_df.sample(n=5)
rand_sample_df

,product_title,product_url,product_breadcrumb,date,price,unit_price,loyalty_price,original_price,store_id,product_category,product_subcategory,product_subcategory_2
product_id,,,,,,,,,,,,
112831044,L'Oreal Men Expert Hydra Energetic Mens With S...,https://groceries.morrisons.com/products/lorea...,toiletries & beauty > mens toiletries > skinca...,2025-02-12,12.0,NaN,NaN,NaN,4,toiletries & beauty,mens toiletries,skincare
8211057,Elfliq Nic Salts Blueberry Sour Rasberry 10ml,https://www.sainsburys.co.uk/gol-ui/product/el...,household > newsagent & tobacconist,2025-02-12,4.0,£4.00 / ea,NaN,4.0,2,household,newsagent & tobacconist,None
112331322,Scholl Shock Reducer Everyday Insoles One Pair,https://groceries.morrisons.com/products/schol...,health > wellbeing & medicines > foot care > i...,2025-02-12,6.0,NaN,NaN,NaN,4,health,wellbeing & medicines,foot care
910002738744,Dishmatic Fill & Clean Heavy Duty Refill Pads,http://groceries.asda.com/product/cleaning/dis...,laundry & household > cleaning > dishwasher & ...,2025-02-12,3.0,50.0p/each,NaN,NaN,3,laundry & household,cleaning,dishwasher & washing up
112016728,Morrisons Brightening Eye Drops,https://groceries.morrisons.com/products/morri...,health > wellbeing & medicines > ear > eye and...,2025-02-12,2.5,NaN,NaN,NaN,4,health,wellbeing & medicines,ear


## Assign gender two those products that mention gender in breadcrumb

In [63]:
# Function to classify gender based on the breadcrumb

def classify_gender(breadcrumb):
    # Define regex patterns for male, female, and neutral categories 
    male_pattern = r'\b(male|men|men\'s|man)\b'
    female_pattern = r'\b(women|female|women\'s|woman|makeup|nailpolish|nail polish)\b' # add makeup and nailpolish as products marketed for women
    
    # Check if the breadcrumb matches the male pattern
    if re.search(male_pattern, breadcrumb, re.IGNORECASE):
        return 'male'
    # Check if the breadcrumb matches the female pattern
    elif re.search(female_pattern, breadcrumb, re.IGNORECASE):
        return 'female'
    # Assign neutral if neither male nor female is found
    else:
        return 'neutral'

In [64]:
df = filtered_items_df.copy()
df.loc[:,'gender_breadcrumb'] = filtered_items_df['product_breadcrumb'].apply(classify_gender)

In [65]:
rand_sample_df = df.sample(n=50)
rand_sample_df

,product_title,product_url,product_breadcrumb,date,price,unit_price,loyalty_price,original_price,store_id,product_category,product_subcategory,product_subcategory_2,gender_breadcrumb
product_id,,,,,,,,,,,,,
317311493,Astonish Specialist Hob Power Cream 500ml,https://www.tesco.com/groceries/en-GB/products...,household > cleaning > hob & oven cleaner > ho...,2025-02-12,2.00,4 / litre,NaN,NaN,1,household,cleaning,hob & oven cleaner,neutral
1000383191195,Rimmel London Lasting Finish 35 Hour Foundatio...,http://groceries.asda.com/product/make-up--nai...,toiletries & beauty > make up & nails > face >...,2025-02-12,10.95,£3.65/10ml,NaN,NaN,3,toiletries & beauty,make up & nails,face,neutral
1000015299693,Tena Men Premium Fit Medium Bladder Weakness P...,http://groceries.asda.com/product/bladder-weak...,toiletries & beauty > bladder weakness > mens ...,2025-02-12,8.50,85.0p/each,NaN,NaN,3,toiletries & beauty,bladder weakness,mens bladder weakness,neutral
436113777302777303,L'Oréal Age Perfect Cell Renew Serum,https://www.waitrose.com/ecom/products/loral-a...,toiletries > health & beauty > facial skincare...,2025-02-12,33.60,£112/100ml,NaN,NaN,8,toiletries,health & beauty,facial skincare,neutral
1000383216855,Herbal Essences Coconut Scent Hydrate Conditio...,http://groceries.asda.com/product/hair-care-dy...,toiletries & beauty > hair care > dye & stylin...,2025-02-12,2.50,£1.25/100ml,NaN,NaN,3,toiletries & beauty,hair care,dye & styling,neutral
67203735,Nelsons Arnicare Arnica Cream 30G,http://groceries.asda.com/product/first-aid/ne...,health & wellness > first aid > antiseptics & ...,2025-02-12,6.00,£20.00/100g,NaN,NaN,3,health & wellness,first aid,antiseptics & insect bite cream,neutral
305040704,Sanex Men Active Control 48 Hr Antiperspirant ...,www.tesco.com/groceries/en-GB/products/305040704,health & beauty > deodorants > antiperspirant ...,2025-02-10,3.00,1.5 / 100ml,2.5,NaN,1,health & beauty,deodorants,antiperspirant,neutral
8158175,Cherry Blossom Round Shoe Lace 140cm,https://www.sainsburys.co.uk/gol-ui/product/ch...,household > household essentials,2025-02-12,2.25,£2.25 / ea,NaN,2.25,2,household,household essentials,None,neutral
8006540109410,"Aussie Miracle Moist Hair Conditioner , Moistu...",https://groceries.aldi.co.uk/en-GB/p-aussie-mi...,health & beauty,2025-02-10,2.99,£1.76 / 100ml,NaN,2.99,5,health & beauty,None,None,neutral


In [66]:
df.shape

(24096, 13)

In [67]:
print(f"Classified as female: {sum(df["gender_breadcrumb"] == "female")/(df.shape[0]):.2%}")
print(f"Classified as male: {sum(df["gender_breadcrumb"] == "male")/(df.shape[0]):.2%}")
print(f"Classified as neutral: {sum(df["gender_breadcrumb"] == "neutral")/(df.shape[0]):.2%}")

Classified as female: 1.91%
Classified as male: 0.02%
Classified as neutral: 98.07%


## Use LLM to create a dictionary for ADM

In [ ]:
# Gendered words dictionary for product packaging
gendered_words = {
    "feminine": [
        "gentle", "soft", "delicate", "radiant", "luminous", "soothing", "dewy",
        "floral", "elegance", "grace", "charm", "blossom", "rose", "pearl", "flower" , "nurturing", "nutritious"
        "for her", "designed for women", "feminine touch" , "volume" , "voluminous", "young" 
    ],
    "masculine": [
        "strong", "rugged", "bold", "powerful", "athletic", "robust", "tough",
        "strength", "force", "cedarwood", "musk", "steel", "for him",
        "engineered for men", "masculine scent", "beast", 
    ],
    "neutral": [
        "fresh", "clean", "natural", "eco-friendly", "sustainable", "modern",
        "quality", "care", "innovation", "balance", "nature"
    ]
}


In [ ]:
female_keywords = {
    "female": {
        "food_beverages": [
            "guilt-free", "nourishing", "wholesome", "superfood", "antioxidant-rich",
            "low-calorie", "dairy-free", "revitalizing", "revitilize", "revitilise","revitilising", "skin-boosting", "detoxifying",
            "pure", "farm-to-table", "probiotic", "vitamin-packed", "heart-healthy",
            "indulgent", "crave-worthy", "mood-enhancing", "meal-prep-friendly"
        ],
        "clothing_apparel": [
            "flattering", "figure", "hugging", "chic", "feminine", "tailored",
            "confidence-boosting", "soft-to-the-touch", "curve-loving", "luxe",
            "couture-inspired", "occasion-ready", "wrinkle-resistant", "seasonless",
            "trend-forward", "size-inclusive"
        ],
        "beauty_skincare": [
            "hydrating", "radiant", "rejuvenating", "age-defying", "anti-aging",
            "glow-boosting", "non-comedogenic", "beauty", "paraben-free",
            "sulfate-free", "luminous", "plumping", "firming", "brightening", "plumed", "clean", "hug", "moisturizing", "moisture", "glitter", "sensitive", 
            "pore-minimizing", "pore-minimising","spa-like", "silky", "velvety", "smudge-proof", "pamper", "young", "elegant", "elegance", 
            "pampering", "clinically proven", "sensitive-skin-friendly", "revitalizing", "revitilize", "revitilise","revitilising", "gentle", "soft", "delicate", "radiant", "luminous", "soothing", "dewy", "floral", "elegance", "grace", "charm", "blossom", "rose", "pearl", "flower" , "nurturing", "nutritious"
        "her","hers", "designed for women", "feminine touch" , "volume" , "voluminous", "young", "spoil", "nourishing", "avocado oil", "radiating", "smooth"
        ],
        "cross_category": [
            "empowering", "curated", "personalized", "transformative", "mindful",
            "bespoke", "award-winning", "iconic", "must-have", "essentials",
            "perfect for you", "perfect for everyday"
        ]
    }
}

male_keywords = {
    "male": {
        "food_beverages": [
            "muscle-fuel", "keto-friendly", "low-carb", "energy-packed", "grass-fed",
            "no-added-sugar", "hearty", "bold flavors", "crafted", "smoked",
            "slow-cooked", "testosterone-supporting", "crunchy", "grillable",
            "fuel-for-the-day"
        ],
        "clothing_apparel": [
            "rugged", "moisture-wicking", "quick-dry", "athletic-fit", "slim-fit",
            "classic-cut", "performance-ready", "stretch-flex", "reinforced",
            "heavy-duty", "weather-resistant", "odour-resistant", "anti-microbial",
            "workwear-inspired", "built-to-last", "multi-pocket", "tech-friendly"
        ],
        "beauty_grooming": [
            "barber-approved", "beard-friendly", "anti-itch", "long-lasting hold",
            "oil-free", "matte finish", "exfoliating", "charcoal-infused",
            "post-shave", "fade-resistant", "strengthening", "travel-sized",
            "tattoo-safe", "sweat-proof"
        ],
        "cross_category": [
            "bold", "no-nonsense", "masculine", "streamlined", "modern",
            "go-anywhere", "built-tough", "precision-engineered", "sleek",
            "mission-critical", "uncompromising", "lean"
        ]
    }
}

neutral_keywords = {
    "neutral": {
        "food_beverages": [
            "sustainably sourced", "organic", "plant-based", "non-GMO", "low-waste",
            "nutrient-dense", "ethically produced", "allergy-friendly", "gluten-free",
            "vegan", "fair-trade", "minimally processed", "whole-food", "balanced",
            "energy-boosting", "immune-supporting", "hydrating", "seasonal",
            "locally grown", "recyclable packaging", "zero-added-sugar", "flexitarian",
            "climate-friendly", "compostable", "mindfully crafted"
        ],
        "clothing_apparel": [
            "unisex", "size-inclusive", "eco-friendly", "versatile", "durable",
            "breathable", "easy-care", "timeless", "minimalist", "adjustable",
            "lightweight", "recycled materials", "modular", "gender-neutral fit",
            "functional", "adaptive", "travel-ready", "certified sustainable",
            "biodegradable", "packable", "long-lasting"
        ],
        "beauty_personal_care": [
            "gentle", "hypoallergenic", "fragrance-free", "cruelty-free", "reef-safe",
            "non-toxic", "pH-balanced", "zero-waste", "dermatologist-tested",
            "sensitive-skin safe", "refillable", "waterless", "carbon-neutral",
            "all-in-one", "travel-friendly", "plastic-free", "universal",
            "transparent ingredients", "ethically sourced", "compostable packaging"
        ],
        "cross_category": [
            "inclusive", "sustainable", "ethical", "innovative", "accessible",
            "adaptable", "transparent", "holistic", "universal", "eco-conscious",
            "future-proof", "community-driven", "thoughtfully designed",
            "quality-crafted", "multi-functional", "user-friendly", "planet-positive",
            "socially responsible"
        ]
    }
}

__Potential challenges__

1. How do we standardize prices to put it on the same scale?

    * option 1: create a categorical variable capturing the type of pricing -- but then how do you convert it into standardized df?

    * option 2 :bring to common denominator as much as possible


2. How do we study the gender price gap?

    * we need to find a way to assign the products into a category (female/male/neither)

3. What method do we study this with?

    * Moshary et al. 2023 use regression


__Pre-processing. Things to do__

- transform data to datetime object


Use one store info to categorize into female/male (Cyril's idea)